In [3]:
# Jupyter Notebook Cell

from itertools import product

def read_input():
    """
    Read lines from input.txt.
    Returns:
        A list of strings (each string is one line of the file).
    """
    with open("input.txt", "r") as f:
        return f.read().splitlines()


def solve_one():
    """
    Solve the puzzle for target=100, max_cheat=2.
    """
    lines = read_input()
    grid = [list(row) for row in lines]
    return cheat(grid, target=100, max_cheat=2)


def solve_two():
    """
    Solve the puzzle for target=100, max_cheat=20.
    """
    lines = read_input()
    grid = [list(row) for row in lines]
    return cheat(grid, target=100, max_cheat=20)


def cheat(grid, target, max_cheat):
    """
    Finds all possible 'cheats' in the grid where the gain is >= target.

    Args:
        grid (list[list[str or int]]): The puzzle grid, where:
          'S' is start
          'E' is end
          '.' is walkable path
          '#' is obstacle
        target (int): The required gain to consider the cheat valid.
        max_cheat (int): Maximum distance (in Manhattan steps) allowed for cheating.

    Returns:
        int: The count of valid cheats that produce a gain >= target.
    """
    base_dist, path, grid = get_base_track(grid)
    result = 0

    for (y, x) in path:
        # If we are close enough to the target so that the required gain is feasible
        if base_dist - grid[y][x] >= target:
            for (cheat_y, cheat_x) in find_cheats(grid, y, x, max_cheat):
                # Calculate the distance and resulting gain
                cheat_dist = abs(cheat_y - y) + abs(cheat_x - x)
                gain = grid[cheat_y][cheat_x] - grid[y][x] - cheat_dist
                if gain >= target:
                    result += 1

    return result


def get_base_track(grid):
    """
    Calculates the base track from 'S' (start) to 'E' (end).

    Returns:
        (int) base_dist: The total distance from start to end (stored at 'E'),
        (list) path: List of (y, x) coordinates representing the path,
        (list[list[int or str]]) grid: The grid with integer distances on the base track.
    """
    sy, sx, ey, ex = get_start_and_end(grid)
    # Convert the start cell to distance 0
    grid[sy][sx] = 0
    path = [(sy, sx)]
    y, x = sy, sx

    dirs = ((0, 1), (1, 0), (0, -1), (-1, 0))

    # Simple BFS-like expansion for the path, building up distance from S
    while (y, x) != (ey, ex):
        for dy, dx in dirs:
            ny, nx = y + dy, x + dx
            if grid[ny][nx] in ['.', 'E']:
                grid[ny][nx] = grid[y][x] + 1
                y, x = ny, nx
                path.append((y, x))
                break

    base_dist = grid[ey][ex]
    return base_dist, path, grid


def get_start_and_end(grid):
    """
    Find coordinates of 'S' (start) and 'E' (end) in the grid.
    """
    h = len(grid)
    w = len(grid[0])

    sy = sx = ey = ex = None

    for y, x in product(range(h), range(w)):
        if grid[y][x] == 'S':
            sy, sx = y, x
        elif grid[y][x] == 'E':
            ey, ex = y, x

    return sy, sx, ey, ex


def find_cheats(grid, y, x, max_cheat):
    """
    Finds all potential cheats around (y, x) within a Manhattan distance of max_cheat.
    """
    h = len(grid)
    w = len(grid[0])
    results = []

    for i, j in product(range(-max_cheat, max_cheat + 1),
                        range(-max_cheat, max_cheat + 1)):
        if abs(i) + abs(j) <= max_cheat:
            endy, endx = y + i, x + j
            # Check boundaries and that it's not an obstacle
            if 0 <= endy < h and 0 <= endx < w and grid[endy][endx] != '#':
                results.append((endy, endx))

    return results

In [4]:
print("Solution to Part One:", solve_one())
print("Solution to Part Two:", solve_two())

Solution to Part One: 1321
Solution to Part Two: 971737
